<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import sys
sys.path.append('../src')
sys.path.append('../utils')

import torch
import time
import numpy as np
from tools import brownian, Lorenz
from scipy.integrate import odeint
from sigKer_torch import SigLoss, SigKernel, SigLoss_naive, SigKernel_naive
import matplotlib.pyplot as plt

# Sig kernel gradients

In [60]:
n=5

A, m, d = 1, 3, 3
X = torch.tensor(Lorenz(A, m), dtype=torch.float64)
Y = torch.tensor(Lorenz(A, m), dtype=torch.float64)
X_ = X.clone()

X.requires_grad = True
X_.requires_grad = True

In [62]:
t = time.time()
ker_naive = SigKernel_naive(X,Y,n)
print('time:', np.round(time.time()-t,3), 's')
print(ker_naive)

time: 3.26 s


In [64]:
t = time.time()
ker = SigKernel.apply(X_,Y,n)
print('time:', np.round(time.time()-t,3), 's')
print(ker)

time: 0.006 s


In [67]:
t = time.time()
ker_naive.backward()
print('time:', np.round(time.time()-t,3), 's')
print(X.grad)

time: 7.032 s
tensor([[[  38.7228,   34.6038, -105.5668],
         [ -32.3123,  -49.7007,   -7.8262],
         [  -6.4105,   15.0969,  113.3930]]], dtype=torch.float64)


In [68]:
# X.grad.data.zero_()
t = time.time()
ker.backward()
print('time:', np.round(time.time()-t,3), 's')
print(X_.grad)

time: 0.001 s
tensor([[[ 31.1878,  27.0379, -88.8584],
         [-28.4850, -45.8999, -16.5074],
         [ -2.7028,  18.8620, 105.3659]]], dtype=torch.float64)


# Sig Loss gradients

In [69]:
n=1

A, m, d = 1, 3, 3
X = torch.tensor(Lorenz(A, m), dtype=torch.float64)
Y = torch.tensor(Lorenz(A, m), dtype=torch.float64)
X_ = X.clone()

X.requires_grad = True
X_.requires_grad = True

In [70]:
t = time.time()
l_naive = SigLoss_naive(n).forward(X,Y)
print('time:', np.round(time.time()-t,3), 's')
print(l_naive)

time: 8.559 s
tensor(24.1579, dtype=torch.float64, grad_fn=<MeanBackward0>)


In [71]:
t = time.time()
l = SigLoss(n).forward(X_,Y)
print('time:', np.round(time.time()-t,3), 's')
print(l)

time: 0.011 s
tensor(24.1579, dtype=torch.float64, grad_fn=<MeanBackward0>)


In [72]:
t = time.time()
l_naive.backward()
print('time:', np.round(time.time()-t,3), 's')
print(X.grad)

time: 15.749 s
tensor([[[-31.0708, -13.9698,  57.6901],
         [ -7.0985,  55.3735,   6.9496],
         [ 38.1693, -41.4038, -64.6398]]], dtype=torch.float64)


In [73]:
X.grad.data.zero_()
t = time.time()
l.backward()
print('time:', np.round(time.time()-t,3), 's')
print(X_.grad)

time: 0.001 s
tensor([[[-26.6542,  -9.0346,  48.9007],
         [-11.1908,  51.6714,  14.9210],
         [ 37.8450, -42.6368, -63.8218]]], dtype=torch.float64)
